In [3]:
## Required packages for calculations and plotting
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [4]:
# Thermal expansion, haline contraction and true scale factor
alph = 1.7657*10**-4
bet = 7.5544*10**-4

volnorming = 10**15 #normalising coeffcients
areanorming = 10**12 #normalising coeffcients
ST_scale=bet/alph

# Establish basic constants 
yr2sec = 365.25*24*60*60
Cp=4000
rho=1024
S0=35

if SF:
    ## ERA5 begins in 1979 so we have to change our "early" and "late" periods
    dyrs = 2014-1979+1
    init_early = 1979
    init_late = 1979
    Early_period = (np.array([init_early,init_early+dyrs]) - init_early)*12
    Late_period = (np.array([init_late,init_late+dyrs]) - init_early)*12
    range_yrs = init_late-init_early+1
else:
    dyrs = 9
    init_early = 1979
    init_late = 2006
    Early_period = (np.array([init_early,init_early+dyrs]) - init_early)*12
    Late_period = (np.array([init_late,init_late+dyrs]) - init_early)*12
    range_yrs = init_late-init_early+1

In [5]:
## LOAD T and S data from a gridded observations (e.g., we use EN4 here)
## For the purposes of the tutorial we only select the first 3 months of the data
data = xr.open_mfdataset('~/UNSW_work/EN4_Data/EN_data/EN4_CT_SA_*')

T = data.Cons_Temp.isel(time=slice(Early_period[0],Early_period[1])).mean('time', keepdims=True)
SST = data.Cons_Temp.isel(time=slice(Early_period[0],Early_period[1])).mean('time', keepdims=True).isel(depth=0)
S = data.Abs_Sal.isel(time=slice(Early_period[0],Early_period[1])).mean('time', keepdims=True)
V = data.dVol.isel(time=slice(Early_period[0],Early_period[1])).mean('time')

In [16]:
if SF:
    EN4_mapped_masks = xr.open_mfdataset('Masks_processed/Masks_EN4_*.nc', combine='nested', concat_dim = 'time')
    fuzz_combined = EN4_mapped_masks.fuzz.isel(time=slice(1,5)).sum('time')/4
else:
    EN4_mapped_masks = xr.open_mfdataset('Masks_processed/Masks_EN4_108*.nc')
mask_EN4_xr = xr.open_mfdataset('Basin_mask/mask_EN4.nc')
mask_EN4 = (mask_EN4_xr.__xarray_dataarray_variable__.values)

EN4_BSP_data = xr.open_mfdataset('BSP_processed/BSP_EN4_TS_*.nc')
Basins = EN4_BSP_data.Basin.values
SA_early =  EN4_BSP_data.S_mean.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')

In [18]:
if SF:
    Opt_result = xr.open_mfdataset('Optimisation_results/Optimal_result_EN4_SF.nc')
else:
    Opt_result = xr.open_mfdataset('Optimisation_results/Optimal_result_EN4.nc')
dT_mixing = Opt_result.dT_mixing.values.reshape(Basins.size,SA_early.shape[-1])
dS_mixing = Opt_result.dS_mixing.values.reshape(Basins.size,SA_early.shape[-1])
dS_adj = Opt_result.dS_adjustment.values.reshape(Basins.size,SA_early.shape[-1])
dT_adj = Opt_result.dT_adjustment.values.reshape(Basins.size,SA_early.shape[-1])

## Remove NaNs from the visualisation
dT_mixing[np.isnan(dT_mixing)] = 0
dS_mixing[np.isnan(dS_mixing)] = 0
dS_adj[np.isnan(dS_adj)] = 0
dT_adj[np.isnan(dT_adj)] = 0

In [33]:
dT_mix_tot = 0
dS_mix_tot = 0
dT_adj_tot = 0
dS_adj_tot = 0
g_ij_tot = 0

for i in tqdm(range(SA_early.shape[-1])):
    tmp_dTmix = 0
    tmp_dSmix = 0
    tmp_dTadj = 0
    tmp_dSadj = 0
    tmp_gij = 0
    for j in range(Basins.size):
        if SF:
            tmp = dT_mixing[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
            tmp_dTmix = tmp+tmp_dTmix
            tmp = dS_mixing[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
            tmp_dSmix = tmp+tmp_dSmix
            tmp = dT_adj[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
            tmp_dTadj = tmp+tmp_dTadj
            tmp = dS_adj[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
            tmp_dSadj = tmp+tmp_dSadj
            tmp = section_trans[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
            tmp_gij = tmp+tmp_gij
        else:
            tmp = dT_mixing[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*EN4_mapped_masks.fuzz[i,:,:,:]
            tmp_dTmix = tmp+tmp_dTmix
            tmp = dS_mixing[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*EN4_mapped_masks.fuzz[i,:,:,:]
            tmp_dSmix = tmp+tmp_dSmix
            tmp = dT_adj[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*EN4_mapped_masks.fuzz[i,:,:,:]
            tmp_dTadj = tmp+tmp_dTadj
            tmp = dS_adj[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*EN4_mapped_masks.fuzz[i,:,:,:]
            tmp_dSadj = tmp+tmp_dSadj
            tmp = section_trans[j,i]*mask_EN4_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
            tmp_gij = tmp+tmp_gij

    dT_mix_tot = tmp_dTmix+dT_mix_tot
    dS_mix_tot = tmp_dSmix+dS_mix_tot
    dT_adj_tot = tmp_dTadj+dT_adj_tot
    dS_adj_tot = tmp_dSadj+dS_adj_tot
    g_ij_tot = tmp_gij+g_ij_tot

  0%|          | 0/128 [00:00<?, ?it/s]

In [42]:
if SF:
    dT_mix_tot.to_netcdf('Remapped_props/dT_mix_Eulerian_SF.nc')
    dS_mix_tot.to_netcdf('Remapped_props/dS_mix_Eulerian_SF.nc')
    dT_adj_tot.to_netcdf('Remapped_props/dT_adj_Eulerian_SF.nc')
    dS_adj_tot.to_netcdf('Remapped_props/dS_adj_Eulerian_SF.nc')
    g_ij_tot.to_netcdf('Remapped_props/g_ij_Eulerian_SF.nc')
else:
    dT_mix_tot.to_netcdf('Remapped_props/dT_mix_Eulerian.nc')
    dS_mix_tot.to_netcdf('Remapped_props/dS_mix_Eulerian.nc')
    dT_adj_tot.to_netcdf('Remapped_props/dT_adj_Eulerian.nc')
    dS_adj_tot.to_netcdf('Remapped_props/dS_adj_Eulerian.nc')
    g_ij_tot.to_netcdf('Remapped_props/g_ij_Eulerian.nc')
